### Prediktiv analyse ved bruk av scikit-learn

Her har vi gjennomført en prediktiv analyse ved hjelp av lineær regresjon for å undersøke fremtidig havnivåstigning. Vi startet med å hente og rense data ved å fjerne feilverdier og interpolere manglende målinger. Deretter forberedte vi datasettet ved å konvertere tid til desimalår og identifiserte gjennomsnittlig havnivå som målvariabel. Regresjonsmodell ble trent med scikit-learn og vurderer nøyaktigheten med MSE og R²-score. Modellen predikere havnivåstigningen frem til det året brukeren ønsker ved å endre variabelen "prediksjonsår" og visualiserte resultatene i en interaktiv graf med Plotly. Dersom brukeren setter prediksjonåret veldig langt inn i fremtiden er det større sjangse for at prediksjonen vil avvike fra virkeligheten.

In [6]:
from pathlib import Path
import sys

# Legg til src/Havnivådata i importstien
project_root = Path.cwd().parents[0]
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Dataprediksjon import HavnivaaPrediksjon

# Kjører modellen, kan endre på slutt_år nedenfor
modell = HavnivaaPrediksjon(slutt_år=2028)
modell.kjør_prediksjon()


c:\Users\madsj\OneDrive\Dokumenter\tdt4114\TDT4114-Prosjekt\src\Havnivådata\Dataprediksjon.py:42: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

c:\Users\madsj\OneDrive\Dokumenter\tdt4114\TDT4114-Prosjekt\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



✅ Modellen er trent. MSE: 34.41, R²: 0.956


In [ ]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from scipy.stats import pearsonr
import plotly.express as px
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Plotly konfig
pio.renderers.default = "vscode"

# Henting av data
response = requests.get(
    "https://sealevel-nexus.jpl.nasa.gov/timeSeriesSpark",
    params={
        "ds": "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly",
        "b": "-180.0,-90.0,180.0,90.0",
        "startTime": "1992-01-01T00:00:00Z",
        "endTime": "2025-01-01T00:00:00Z",
        "output": "JSON"
    }
)

# Konverter JSON til DataFrame
if response.ok and "data" in response.json():
    raw_data = [item for sublist in response.json()["data"] for item in sublist]
    df = pd.DataFrame(raw_data)
else:
    raise ValueError("Feil ved henting av data.")

# Formater tid og enheter
df["iso_time"] = pd.to_datetime(df["iso_time"])
df["år"] = df["iso_time"].dt.year
df["måned"] = df["iso_time"].dt.to_period("M")
df["mean_mm"] = df["mean"] * 1000
df["min_mm"] = df["min"] * 1000
df["max_mm"] = df["max"] * 1000

# Rensing
grenser = {
    "mean": (-1, 1),
    "min": (-1, 1),
    "max": (-1, 0.6)
}
ugyldig = pd.Series(False, index=df.index)
for kolonne, (min_grense, max_grense) in grenser.items():
    ugyldig |= ~df[kolonne].between(min_grense, max_grense)
df.loc[ugyldig, ["mean_mm", "min_mm", "max_mm"]] = np.nan
df[["mean_mm", "min_mm", "max_mm"]] = df[["mean_mm", "min_mm", "max_mm"]].interpolate(method="linear")
df_renset = df.copy()

# === LINEÆR REGRESJON OG PREDIKSJON ===

# Forbered data
df_renset["år_decimal"] = df_renset["iso_time"].dt.year + df_renset["iso_time"].dt.month / 12
X = df_renset[["år_decimal"]].values
y = df_renset["mean_mm"].values

# Tren modellen
modell = LinearRegression()
modell.fit(X, y)

# Evaluer
y_pred = modell.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"Modellen er trent. MSE: {mse:.2f}, R²: {r2:.3f}")

# Variabel for hvor lenge vi skal predikere 
prediksjonsår = 2100

# Fremtidig prediksjon til det angitte året
framtid = pd.DataFrame({
    "år_decimal": np.arange(2025, prediksjonsår + 1, 1/12)
})
framtid["mean_mm_pred"] = modell.predict(framtid[["år_decimal"]])

# Visualisering med Plotly
fig = px.line(
    df_renset, 
    x="år_decimal", 
    y="mean_mm", 
    title="Lineær regresjon: Gjennomsnittlig havnivåstigning over tid", 
    labels={"år_decimal": "År", "mean_mm": "Havnivå (mm)"},
    line_shape='linear'
)

fig.add_scatter(
    x=framtid["år_decimal"], 
    y=framtid["mean_mm_pred"], 
    mode="lines", 
    name=f"Predikert til {prediksjonsår}", 
    line=dict(color="red")
)

fig.update_layout(
    title="Lineær regresjon: Gjennomsnittlig havnivåstigning over tid",
    xaxis_title="År",
    yaxis_title="Havnivå (mm)",
    showlegend=True
)

fig.show()


C:\Users\madsj\AppData\Local\Temp\ipykernel_28056\2606320175.py:39: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

c:\Users\madsj\OneDrive\Dokumenter\tdt4114\TDT4114-Prosjekt\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



Modellen er trent. MSE: 34.41, R²: 0.956
